In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import dask.dataframe as dd
import dask.array as da
import os
from PIL import Image

In [2]:
def path_to_pixels(image_path):
    im = Image.open(image_path)
    resized_image = im.resize((256, 256))
    return np.array(resized_image.getdata()).reshape((256, 256, 3))

In [3]:
X = []
y = []

In [4]:
# fake = 1
# real = 0

In [5]:
real_img_path = '/content/real_and_fake_face/training_real'
fake_img_path = '/content/real_and_fake_face/training_fake'

for img in os.listdir(real_img_path):
  if img.endswith(".jpg"):
    img_path = os.path.join(real_img_path, img)
    X.append(path_to_pixels(img_path))
    y.append(0)

for img in os.listdir(fake_img_path):
  if img.endswith(".jpg"):
    img_path = os.path.join(fake_img_path, img)
    X.append(path_to_pixels(img_path))
    y.append(1)

In [6]:
X = np.array(X)
y = np.array(y)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model_base = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=[256, 256, 3],
    pooling=None,
    classes=2,
    classifier_activation='softmax'
)

In [9]:
for layer in model_base.layers:
    layer.trainable = False

model = keras.models.Sequential()
model.add(model_base)
model.add(keras.layers.AveragePooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))


opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics = ["accuracy", tf.metrics.AUC()])

In [11]:
epochs = 5
bs = 64
model.fit(
    x=X_train,
    y=y_train,
    batch_size=bs,
    epochs=epochs
)

Epoch 1/5
26/26 [==============================] - 32s 482ms/step - loss: 109.1009 - accuracy: 0.4908 - auc: 0.4887
Epoch 2/5
26/26 [==============================] - 10s 399ms/step - loss: 24.5467 - accuracy: 0.5288 - auc: 0.5238
Epoch 3/5
26/26 [==============================] - 10s 402ms/step - loss: 13.3057 - accuracy: 0.5331 - auc: 0.5353
Epoch 4/5
26/26 [==============================] - 11s 409ms/step - loss: 14.4859 - accuracy: 0.5031 - auc: 0.5043
Epoch 5/5
26/26 [==============================] - 11s 411ms/step - loss: 23.0033 - accuracy: 0.5184 - auc: 0.5183


In [12]:
model.evaluate(X_test, y_test)

13/13 [==============================] - 9s 374ms/step - loss: 15.2160 - accuracy: 0.5306 - auc: 0.4986


[15.216014862060547, 0.5305623412132263, 0.4986070394515991]

In [13]:
epochs = 20
bs = 64
model.fit(
    x=X_train,
    y=y_train,
    batch_size=bs,
    epochs=epochs
)

Epoch 1/20
26/26 [==============================] - 11s 414ms/step - loss: 12.3405 - accuracy: 0.5423 - auc: 0.5424
Epoch 2/20
26/26 [==============================] - 11s 418ms/step - loss: 19.9036 - accuracy: 0.5270 - auc: 0.5269
Epoch 3/20
26/26 [==============================] - 11s 423ms/step - loss: 23.7800 - accuracy: 0.5325 - auc: 0.5325
Epoch 4/20
26/26 [==============================] - 11s 425ms/step - loss: 13.0659 - accuracy: 0.5172 - auc: 0.5223
Epoch 5/20
26/26 [==============================] - 11s 429ms/step - loss: 7.0102 - accuracy: 0.5741 - auc: 0.5871
Epoch 6/20
26/26 [==============================] - 11s 430ms/step - loss: 10.5706 - accuracy: 0.5337 - auc: 0.5378
Epoch 7/20
26/26 [==============================] - 11s 426ms/step - loss: 24.1821 - accuracy: 0.5165 - auc: 0.5113
Epoch 8/20
26/26 [==============================] - 11s 425ms/step - loss: 7.6738 - accuracy: 0.5631 - auc: 0.5696
Epoch 9/20
26/26 [==============================] - 11s 423ms/step - loss:

In [14]:
model.evaluate(X_test, y_test)

13/13 [==============================] - 3s 222ms/step - loss: 26.5009 - accuracy: 0.4670 - auc: 0.5000


[26.500917434692383, 0.46699267625808716, 0.5]